In [426]:
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (
    NoiseModel,
    pauli_error,
)

import os
import pickle
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [427]:
class SecureChannel:
    def __init__(self, key: bytes):
        """
        Initialize the secure channel with a shared symmetric key.
        Uses AES-GCM for both encryption (confidentiality) and authentication.
        """
        self.key = key
        self.aesgcm = AESGCM(key)
    
    def send(self, plaintext: bytes, associated_data: bytes = None):
        """
        Encrypts the plaintext using AES-GCM.
        Generates a new random 12-byte nonce for every encryption.
        Optionally authenticates the provided associated_data.
        Returns a tuple (nonce, ciphertext).
        """
        nonce = os.urandom(12)  # 96-bit nonce typical for AES-GCM.
        ciphertext = self.aesgcm.encrypt(nonce, plaintext, associated_data)
        return nonce, ciphertext
    
    def receive(self, nonce: bytes, ciphertext: bytes, associated_data: bytes = None):
        """
        Decrypts the provided ciphertext using AES-GCM and the given nonce.
        If the ciphertext or associated data has been tampered with,
        the decryption fails and an error is raised.
        """
        try:
            plaintext = self.aesgcm.decrypt(nonce, ciphertext, associated_data)
            return plaintext
        except Exception as e:
            print("Decryption failed or authentication error:", e)
            return None

    def send_object(self, obj, associated_data: bytes = None):
        """
        Serializes the given object to bytes using pickle, and then encrypts it.
        Returns the nonce and ciphertext.
        """
        obj_bytes = pickle.dumps(obj)
        return self.send(obj_bytes, associated_data)

    def receive_object(self, nonce: bytes, ciphertext: bytes, associated_data: bytes = None):
        """
        Decrypts the ciphertext to obtain the serialized object bytes, and then
        deserializes it using pickle.
        Returns the original Python object if successful, thierwise None.
        """
        obj_bytes = self.receive(nonce, ciphertext, associated_data)
        if obj_bytes is None:
            return None
        return pickle.loads(obj_bytes)

In [428]:
class Participant:
    def __init__(self, with_error: bool = True, classical_error: float = 1e-5):
        # everything we might send or receive like the measurement bases
        self.data = {"my_bases": []}
        # raw/sifted/filtered keys
        self.raw_key = []
        self.sifted_key = []
        self.filtered_key = np.array([])
        self.final_key = []
        # secure channel placeholders; set up by Sender or Receiver.accept_secure_channel
        self.secure_channel = None
        self.associated_data = b"BB84 Secure Channel Integrity"
        # pre-shared random seed
        self.key_arrangement = []

        # error
        self.with_error = with_error
        self.classical_error = classical_error

    def _apply_classical_noise(self, bits: list[int]) -> list[int]:
        flips = np.random.rand(len(bits)) < self.classical_error
        return [b ^ int(f) for b, f in zip(bits, flips)]

    def send(self, data_key: str):
        """Encrypt and send whatever is in self.data[data_key]."""
        payload = self.data[data_key]
        # only if it's truly a bit-list
        if self.with_error and isinstance(payload, list) and payload and sum(payload) <= len(payload) and data_key != "my_basis": # assuming bases exchange step is error free to avoid index out of range error
           noisy = self._apply_classical_noise(payload)
        else:
           noisy = payload
        return self.secure_channel.send_object(noisy, self.associated_data)

    def receive(self, data_key: str, encrypted):
        """Decrypt and store into self.data[data_key]."""
        nonce, ciphertext = encrypted
        self.data[data_key] = self.secure_channel.receive_object(
            nonce, ciphertext, self.associated_data
        )

    def sift_key(self, my_bases_key: str = "my_bases", their_bases_key: str = "thier_bases"):
        """Keep only those raw_key bits where our bases == theirs."""
        my_bases = self.data[my_bases_key]
        thier_bases = self.data[their_bases_key]
        self.sifted_key = [
            self.raw_key[i]
            for i in range(len(self.raw_key))
            if my_bases[i] == thier_bases[i]
        ]

    def calculate_verification_set(self, verification_indices_key: str = "verification_indices", verification_set_key: str = "verification_set"):
        idxs = self.data[verification_indices_key]
        self.data[verification_set_key] = [
            self.sifted_key[i] for i in idxs
        ]

    def check_verification_sets(self, verification_set_key: str = "verification_set", thier_verification_set_key: str = "thier_verification_set"):
        verification_set = self.data[verification_set_key]
        thier_verification_set = self.data[thier_verification_set_key]
        mismatches = 0
        length = len(verification_set)
        for i in range(length):
            mismatches = mismatches + 1 if verification_set[i] != thier_verification_set[i] else mismatches
        return mismatches/length # error rate
    
    def filter_sifted_key(self, verification_indices_key: str = "verification_indices"):
        exposed_bits = self.data[verification_indices_key]
        self.filtered_key = np.array([self.sifted_key[i] for i in range(len(self.sifted_key)) if not i in exposed_bits])
        self.key_arrangement = [i for i in range(len(self.filtered_key))]
    
    def parity(self, block):
        return sum(block)%2
    
    def new_key_arrangement(self):
        self.rng.shuffle(self.key_arrangement)

    def calculate_blocks_parities(self, block_size, shuffle = True, parities_key: str = "parities"):
        self.data[parities_key] = []
        if shuffle: self.new_key_arrangement()
        shuffled_filtered_key = self.filtered_key[self.key_arrangement]
        n = len(shuffled_filtered_key)
        i = 0
        for i in range(0,n//block_size):
            self.data[parities_key].append(self.parity(shuffled_filtered_key[i*block_size:(i+1)*block_size]))
        if (i+1)*block_size < n:
            self.data[parities_key].append(self.parity(shuffled_filtered_key[(i+1)*block_size:]))
        
    def calculate_parity(self, shuffle = True, parities_key: str = "parities", parity_indices_key: str = "parity_indices"):
        self.data[parities_key] = []
        starting_index, ending_index = self.data[parity_indices_key]
        if shuffle: self.new_key_arrangement()
        shuffled_filtered_key = self.filtered_key[self.key_arrangement]
        n = len(shuffled_filtered_key)
        if starting_index >= 0 and ending_index >= 0:
            self.data[parities_key] = self.parity(shuffled_filtered_key[starting_index:ending_index])

    def reset_data(self):
        # everything we might send or receive like the measurement bases
        self.data = {"my_bases": []}
        # raw/sifted/filtered keys
        self.raw_key = []
        self.sifted_key = []
        self.filtered_key = np.array([])
        # secure channel placeholders; set up by Sender or Receiver.accept_secure_channel
        self.secure_channel = None
        # pre-shared random seed
        self.key_arrangement = []


    def toeplitz_privacy_amplify(self,total_bits_leaked: int, lambda_sec: int = 32):
        """
        filtered_key: array of 0/1 of length n
        final_len: desired key length k (< n)
        rng: RandomState known to both parties
        """
        n = len(self.filtered_key)
        k = n - total_bits_leaked - lambda_sec 
        # 1) draw a random seed of length n + k - 1
        seed = self.rng.randint(0, 2, size=(n + k - 1,), dtype=int)

        # 2) construct each of the k Toeplitz rows on the fly:
        self.final_key = np.zeros(k, dtype=int)
        for i in range(k):
            # row i is seed[i : i + n]
            row = seed[i : i + n]
            # dot-product mod 2
            self.final_key[i] = (row @ self.filtered_key) & 1

In [429]:
class Sender(Participant):
    def __init__(self, s: int, with_error: bool = True, classical_error: int = 1e-5):
        super().__init__(with_error=with_error,classical_error=classical_error)
        self.s = s

    def start_secure_channel(self):
        # set up secure channel
        key = AESGCM.generate_key(bit_length=128)
        self.secure_channel = SecureChannel(key)
        # reproducible randomness
        self.rng = np.random.RandomState(key[len(key)//2])
        return (self.secure_channel, self.associated_data)

    def prepare_qubit(self, my_bases_key: str = "my_bases"):
        qc = QuantumCircuit(1, 1)
        # random bit and bases
        a = np.random.randint(0, 2)
        b = np.random.randint(0, 2)
        if a: qc.x(0)
        if b: qc.h(0)
        self.qubit = qc
        self.raw_key.append(a)
        self.data[my_bases_key].append(b)

    def send_qubit(self):
        return self.qubit

    def generate_verification_indices(self, verification_indices_key: str = "verification_indices"):
        n = len(self.sifted_key)
        self.data[verification_indices_key] = random.sample(range(n), k=self.s)

In [430]:
class Receiver(Participant):
    def __init__(self, with_error = True, measurement_error = 1e-2, gate_error = 5e-2, classical_error: int = 1e-5):
        super().__init__(with_error=with_error,classical_error=classical_error)

        # self.measurement_error = measurement_error
        # self.gates_error = gates_error
        noise_model = None
        if with_error:
            # QuantumError objects
            error_meas = pauli_error([("X", measurement_error), ("I", 1 - measurement_error)])
            error_gate1 = pauli_error([("X", gate_error), ("I", 1 - gate_error)])

            # Add errors to noise model
            noise_model = NoiseModel()
            noise_model.add_all_qubit_quantum_error(error_meas, "measure")
            noise_model.add_all_qubit_quantum_error(error_gate1, ["x", "h"])

        self.simulator = AerSimulator(noise_model=noise_model)

    def accept_secure_channel(self, channel_params):
        self.secure_channel, self.associated_data = channel_params
        key = self.secure_channel.key
        self.rng = np.random.RandomState(key[len(key)//2])

    def receive_qubit(self, qc: QuantumCircuit):
        self.qubit = qc

    def prepare_and_measure(self, my_bases_key: str = "my_bases"):
        # choose bases
        b = np.random.randint(0, 2)
        if b:
            self.qubit.h(0)
        self.data[my_bases_key].append(b)
        # measure
        self.qubit.measure(0, 0)
        result = self.simulator.run(self.qubit, shots=1).result()
        counts = result.get_counts()
        zeros, ones =  counts["0"] if '0' in counts else -1, counts["1"] if '1' in counts else -1
        outcome = 0 if zeros > ones else 1
        self.raw_key.append(outcome)

    def different_parity_blocks(self, parities_key: str = "parities", thier_parities_key: str = "thier_parities"):
        different_blocks = []
        parities = self.data[parities_key]
        thier_parities = self.data[thier_parities_key]
        n = len(parities)
        for i in range(n):
            if parities[i] != thier_parities[i]:
                different_blocks.append(i)
        return different_blocks
    
    def generate_parity_indices(self, starting_index: int, ending_index: int, parity_indices_key: str = "parity_indices"):
        self.data[parity_indices_key] = [starting_index, ending_index]
    
    def is_same_parity(self, parities_key: str = "parities", thier_parities_key: str = "thier_parities"):
        return self.data[parities_key] == self.data[thier_parities_key]
    
    def correct_bits(self, bit_indices):
        # bit_indices are positions *in the shuffled* key
        for position in bit_indices:
            original_position = self.key_arrangement[position]
            # flip 0↔1 by subtraction
            self.filtered_key[original_position] = 1 - int(self.filtered_key[original_position])



In [431]:
class Eavesdropper(Receiver):
    def __init__(self,  with_error = False, measurement_error = 1e-2, gate_error = 5e-2, classical_error = 1e-5):
        super().__init__(with_error, measurement_error, gate_error, classical_error)

In [432]:
class CascadeCorrection():
    def __init__(self, sender: Sender, receiver: Receiver):
        self.sender = sender
        self.receiver = receiver

    def parity(block):
        return sum(block)%2
    
    def binary_search_correction(self, starting_index, ending_index):
        if ending_index - starting_index == 1:
            return starting_index
            
        mid = (starting_index+ending_index) // 2

        self.receiver.generate_parity_indices(starting_index=starting_index, ending_index=mid, parity_indices_key="parity_indices")
        self.receiver.calculate_parity(shuffle=False, parities_key="parities", parity_indices_key="parity_indices")
        enc = self.receiver.send("parity_indices")
        self.sender.receive("parity_indices", enc)
        self.sender.calculate_parity(shuffle=False, parities_key="parities", parity_indices_key="parity_indices")
        enc = self.sender.send("parities")
        self.receiver.receive("thier_parities", enc)
        if not self.receiver.is_same_parity(): 
            return self.binary_search_correction(starting_index=starting_index,ending_index=mid)
        
        else:
            self.receiver.generate_parity_indices(starting_index=mid, ending_index=ending_index, parity_indices_key="parity_indices")
            self.receiver.calculate_parity(shuffle=False, parities_key="parities", parity_indices_key="parity_indices")
            enc = self.receiver.send("parity_indices")
            self.sender.receive("parity_indices", enc)
            self.sender.calculate_parity(shuffle=False, parities_key="parities", parity_indices_key="parity_indices")
            enc = self.sender.send("parities")
            self.receiver.receive("thier_parities", enc)
            return self.binary_search_correction(starting_index=mid,ending_index=ending_index)

            

    def run(self, block_size: int, correction_passes_per_block_size: int = 4):
        n = len(self.receiver.filtered_key)
        total_bits_leaked = 0
        while block_size >= 1:
            for _ in range(correction_passes_per_block_size):
                bits_to_correct = set()
                self.sender.calculate_blocks_parities(block_size, shuffle=True)
                self.receiver.calculate_blocks_parities(block_size, shuffle=True)
                enc = self.sender.send("parities")
                self.receiver.receive("thier_parities",enc)
                different_blocks = self.receiver.different_parity_blocks(parities_key="parities", thier_parities_key="thier_parities")
                starting_index = -1
                ending_index = -1
                for i in range(len(different_blocks)):
                    starting_index = different_blocks[i]*block_size
                    ending_index = min(starting_index + block_size, n)
                    bits_to_correct.add(self.binary_search_correction(starting_index,ending_index))
                total_bits_leaked += len(bits_to_correct)
                self.receiver.correct_bits(bits_to_correct)
            block_size//=2

        return total_bits_leaked


In [433]:
class BB84:
    def __init__(
        self,
        gate_error: float = 1e-2,
        measurement_error: float = 5e-2,
        classical_error: float = 1e-5,
        false_alarm: float = 1e-2,
        error_margin: float = 5e-2,
        block_size: int = 256,
        correction_passes_per_block_size: int = 4,
        with_eavesdropper: bool = False,
        with_error: bool = True,
        with_error_correction: bool = True
    ):
        # self.gate_error = gate_error
        # self.measurement_error = measurement_error
        # self.classical_error = classical_error
        # self.false_alarm = false_alarm
        # self.error_margin = error_margin
        self.block_size = block_size
        self.with_eavesdropper = with_eavesdropper
        self.with_error = with_error
        self.with_error_correction = with_error_correction
        
        p_gates_error = 0.5*(1-sum([(1-2*gate_error)**i for i in range(4)])/4)

        pQBER = (1-(1-2*p_gates_error)*(1-2*measurement_error)*(1-2*classical_error))/2
        self.s = int(np.ma.log(1/false_alarm)//(2*error_margin**2))
        self.n = int(2 ** (int(np.log2(self.s)) + 4))
        ds = np.ma.sqrt(np.ma.log(1/false_alarm)/(2*self.s))
        self.error_threshold = pQBER + ds

        self.correction_passes_per_block_size = correction_passes_per_block_size

        self.alice = Sender(classical_error=classical_error, s=self.s)
        self.bob   = Receiver(classical_error=classical_error,with_error=with_error,gate_error=gate_error,measurement_error=measurement_error)
        self.trudy = Eavesdropper()

    def run(self):
        intercepted = True
        while intercepted:
            # 1) Quantum transmission
            for _ in range(self.n):
                self.alice.prepare_qubit()
                q = self.alice.send_qubit()
                if self.with_eavesdropper:
                    self.trudy.receive_qubit(q)
                    self.trudy.prepare_and_measure()
                self.bob.receive_qubit(q)
                self.bob.prepare_and_measure()
            
            self.with_eavesdropper = False


            # 2) Establish secure channel
            channel_params = self.alice.start_secure_channel()
            self.bob.accept_secure_channel(channel_params)

            # 3) bases exchange
            enc = self.alice.send("my_bases")
            self.bob.receive("thier_bases", enc)

            enc = self.bob.send("my_bases")
            self.alice.receive("thier_bases", enc)

            # 4) Sifting
            self.alice.sift_key(my_bases_key="my_bases", their_bases_key="thier_bases")
            self.bob.sift_key(my_bases_key="my_bases", their_bases_key="thier_bases")

            # 5) Verification
            self.alice.generate_verification_indices(verification_indices_key="verification_indices")
            enc = self.alice.send("verification_indices")
            self.bob.receive("verification_indices", enc)

            self.alice.calculate_verification_set(verification_indices_key="verification_indices", verification_set_key="verification_set")
            self.bob.calculate_verification_set(verification_indices_key="verification_indices", verification_set_key="verification_set")

            enc = self.alice.send("verification_set")
            self.bob.receive("thier_verification_set", enc)

            enc = self.bob.send("verification_set")
            self.alice.receive("thier_verification_set", enc)

            alice_error_rate = self.alice.check_verification_sets(verification_set_key="verification_set", thier_verification_set_key="thier_verification_set")
            bob_error_rate = self.bob.check_verification_sets(verification_set_key="verification_set", thier_verification_set_key="thier_verification_set")

            intercepted = alice_error_rate > self.error_threshold or bob_error_rate > self.error_threshold
            
            # 6) Filtering the verification set from the sifted key
            if not intercepted:
                self.alice.filter_sifted_key(verification_indices_key="verification_indices")
                self.bob.filter_sifted_key(verification_indices_key="verification_indices")
                print("Alice sifted key:   ", self.alice.sifted_key)
                print("Bob   sifted key:   ", self.bob.sifted_key)
                print("Alice verify set:   ", self.alice.data["verification_set"])
                print("Bob   verify set:   ", self.bob.data["verification_set"])
                if self.with_error_correction:
                    print("Starting error correction...")
                    correction = CascadeCorrection(self.alice, self.bob)
                    total_bits_leaked = correction.run(self.block_size, self.correction_passes_per_block_size)
                    print(f"{total_bits_leaked} bits have been revealed during the correction")
                    self.alice.toeplitz_privacy_amplify(total_bits_leaked=total_bits_leaked, lambda_sec=32)
                    self.bob.toeplitz_privacy_amplify(total_bits_leaked=total_bits_leaked, lambda_sec=32)
                else:
                    print("No need to apply toeplitz privacy amplify since no bits have been leaked in to perform error correction. So, filtered key is the final key.")

                print("Alice filtered key:   ", self.alice.filtered_key.tolist())
                print("Bob   filtered key:   ", self.bob.filtered_key.tolist())
                print("Alice's and Bob's filtered keys are the same:",np.array_equal(self.alice.filtered_key, self.bob.filtered_key))
                print(f"The raw key consists of {self.n} bits.")
                print(f"The sifted key consists of {len(self.alice.sifted_key)} bits.")
                print(f"The verification set consists of {self.s} bits.")
                print(f"The filtered key consists of {len(self.alice.filtered_key)} bits.")

                if self.with_error_correction:
                    print(f"Final key consists of {len(self.alice.final_key)} bits.")
                    print("Alice final key:   ", self.alice.final_key.tolist())
                    print("Bob   final key:   ", self.bob.final_key.tolist())
                    print("Alice's and Bob's final keys are the same:",np.array_equal(self.alice.final_key, self.bob.final_key))
            
            else:
                self.alice.reset_data()
                self.bob.reset_data()
                print("Someone else has measured the qubits.")
                print("Starting again without eavesdropper...")

            # 7) Check
            print("Verification sets mismatches is less than error threshold:", not intercepted)
            print(f"Alice calculated the mismatching rate between the verification set to be: {alice_error_rate*100}% and the error threshold is: {self.error_threshold*100}%")
            print(f"Bob calculated the mismatching rate between the verification set to be: {bob_error_rate*100}% and the error threshold is: {self.error_threshold*100}%")

# BB84 With no evesdropper and no error

In [434]:
bb84 = BB84(with_eavesdropper=False, with_error=False, with_error_correction=False)
bb84.run()

Alice sifted key:    [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 

# BB84 With evesdropper and no error

In [435]:
bb84 = BB84(with_eavesdropper=True, with_error=False, with_error_correction=False)
bb84.run()

Someone else has measured the qubits.
Starting again without eavesdropper...
Verification sets mismatches is less than error threshold: False
Alice calculated the mismatching rate between the verification set to be: 36.91639522258415% and the error threshold is: 11.33305574930518%
Bob calculated the mismatching rate between the verification set to be: 36.91639522258415% and the error threshold is: 11.33305574930518%
Alice sifted key:    [0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 

# BB84 With error and no evesdropper

## Without error correction

In [436]:
bb84 = BB84(with_eavesdropper=False, with_error=True, with_error_correction=False)
bb84.run()

Alice sifted key:    [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 

## With error correction

In [437]:
bb84 = BB84(with_eavesdropper=False, with_error=True, with_error_correction=True)
bb84.run()

Someone else has measured the qubits.
Starting again without eavesdropper...
Verification sets mismatches is less than error threshold: False
Alice calculated the mismatching rate between the verification set to be: 28.121606948968513% and the error threshold is: 11.33305574930518%
Bob calculated the mismatching rate between the verification set to be: 28.121606948968513% and the error threshold is: 11.33305574930518%
Alice sifted key:    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0

# BB84 With error and evesdropper

## Without error correction

In [438]:
bb84 = BB84(with_error=True, with_eavesdropper=True, with_error_correction= False)
bb84.run()

Someone else has measured the qubits.
Starting again without eavesdropper...
Verification sets mismatches is less than error threshold: False
Alice calculated the mismatching rate between the verification set to be: 38.762214983713356% and the error threshold is: 11.33305574930518%
Bob calculated the mismatching rate between the verification set to be: 38.762214983713356% and the error threshold is: 11.33305574930518%
Alice sifted key:    [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0

## With error correction

In [439]:
bb84 = BB84(with_error=True, with_eavesdropper=True, with_error_correction=True)
bb84.run()

Someone else has measured the qubits.
Starting again without eavesdropper...
Verification sets mismatches is less than error threshold: False
Alice calculated the mismatching rate between the verification set to be: 39.08794788273616% and the error threshold is: 11.33305574930518%
Bob calculated the mismatching rate between the verification set to be: 39.08794788273616% and the error threshold is: 11.33305574930518%
Alice sifted key:    [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 